# [Module 2.0] Inferencde Pipeline 생성

- 이 노트북에서는 아래의 내용을 진행 합니다.
    - Feature Transfomer(전처리 학습 모델) 생성
    - Train 데이타를 Feature Transfomer를 통해서 전처리 데이타 생성
    - Validation 데이타를 Feature Transfomer를 통해서 전처리 데이타 생성
    - XGBoost를 학습
    - Post-Processing Model 생성
    - Inference Pipeline 생성
- 소요 시간은 약 10분 걸립니다.

## Feature Transformer (전처리 학습 모델) - preprocessing.py 파일
- Numerical 데이타는 <a href=https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html>StandardScaler</a>를 사용하여 Normalization을 함. 
    * z = (x - u) / s 
        * (z: 표준화된 값. 이 값을 학습시에 사용, x: 각 테이타의 값, u: 데이타 항목의 평균, s: 데이타 항목의 표준편차)
- 아래 Account Length, ..CustServ Calls까지 모두 위의 방법으로 전처리 함.
    - 아래 imputer는 결측값이 있을 경우에 해당 컬럼의 median 값을 사용 함.

```python
    numeric_features = list([
    'Account Length',
    'VMail Message',
    'Day Mins',
    'Day Calls',
    'Eve Mins',
    'Eve Calls',
    'Night Mins',
    'Night Calls',
    'Intl Mins',
    'Intl Calls',
    'CustServ Calls'])

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
```
- Categorical 데이타는 One Hot Encoding 방식으로 전처리 함. (예, 남자:0, 여자:1 일 경우에 남자:(1,0), 여자:(0,1) 방식으로 처리)
    - State, Area Code, Int'l Plan, VMail Plan을 적용 함
```python
    categorical_features = ['State','Area Code',"Int'l Plan",'VMail Plan']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
```
- 최종적으로 Numerical and Categorical Transformer를 합쳐서 Transformer 생성하고, 학습하여 Transformer의 모델을 S3에 업로드 함.
```python
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)],
        remainder="drop")

    preprocessor.fit(concat_data)

    joblib.dump(preprocessor, os.path.join(args.model_dir, "model.joblib"))
```
- Phone 데이타 항목은 위의 전처리 항목에서 제외 함. 유저별로 고유한 번호이기에 피쳐로서 의미가 없을 것으로 보임

In [1]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [2]:
%store -r

## Feature Transformer (전처리 학습 모델) 생성
아래는 다음과 같은 작업을 합니다.
- SKLearn 이라는 Estimator를 생성 합니다. 
    - s3_input_train의 학습 데이타를 SKLearn 입력으로 제공 합니다.
    - "전처리 학습 모델 (Featurizer)" 을 생성할 수 있는 소스 코드 preprocessing.py 를 지정 합니다. 
    - 사용할 리소스로 instance_type = 'local' 를 지정 합니다. (이미 노트북 인스턴스에 설치된 Docker-compose를 이용 합니다.)
        - **Local 이 아니라 SageMaker Cloud Instance도 사용 가능 합니다. (예: ml.m4.xlarge)**
        - **아래 XGBoost 학습 알고리즘을 사용시에는 SageMaker Cloud Instance 사용함**
- SKLearn의 "전처리 학습 모델"이 완료가 되면 결과인 모델 아티펙트 파일이 (model.tar.gz)  s3://{bucket_name}/{job_name}/output.tar.gz 에 저장 됩니다. 
    - (예: s3://sagemaker-us-east-2-057716757052/sagemaker-scikit-learn-2020-07-15-08-39-41-035/model.tar.gz)

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [3])로 바뀔 때까지 기다려 주세요

In [3]:
from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
from sagemaker import get_execution_role

role = get_execution_role()

script_path = 'train-preprocessing.py'
# instance_type = 'ml.m4.2xlarge'
instance_type = 'local'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    train_instance_type = instance_type
)
sklearn_preprocessor.fit({'train': s3_input_train})

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


Creating tmphxbdmdkq_algo-1-6f0de_1 ... 
Attaching to tmphxbdmdkq_algo-1-6f0de_12mdone
algo-1-6f0de_1  | 2020-08-09 08:00:55,118 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-6f0de_1  | 2020-08-09 08:00:55,120 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-6f0de_1  | 2020-08-09 08:00:55,129 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-6f0de_1  | 2020-08-09 08:00:55,235 sagemaker-containers INFO     Module train-preprocessing does not provide a setup.py. 
algo-1-6f0de_1  | Generating setup.py
algo-1-6f0de_1  | 2020-08-09 08:00:55,235 sagemaker-containers INFO     Generating setup.cfg
algo-1-6f0de_1  | 2020-08-09 08:00:55,236 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-6f0de_1  | 2020-08-09 08:00:55,236 sagemaker-containers INFO     Installing module with the following command:
algo-1-6f0de_1  | /miniconda3/bin/python -m pip install . 
algo-1-6f0de_1 

## Feature Transfomer를 사용하여 전처리된 학습 및 검증 데이타 생성 

![Transformer_Train](img/Fig2.1.transformer_train.png)

### Preprocessed Training data (Feature) 만들기

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [4])로 바뀔 때까지 기다려 주세요

In [4]:
# 아웃풋 경로 지정
transform_train_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-train-output')
instance_type = 'local'
# instance_type = 'ml.m4.2xlarge'

# scikit_learn_inferencee_model 이름으로 전처리 학습 모델 생성
# TRANSFORM_MODE의 환경 변수는 전처리 모드라는 것을 알려 줌.
    # 추론시에는 환경 변수를 TRANSFORM_MODE": "inverse-label-transform" 설정 함.
    # 위의 두개의 과정을 분리할 수 있으나, 한개의 소스를 (preprocessor.py)를 사용하기 위해서, 환경 변수를 통해서 구분함.
scikit_learn_inferencee_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})
# scikit_learn_inferencee_model 에서 Train Transformer 생성
transformer_train = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_train_output_path,
    accept = 'text/csv')


# Preprocess training input
transformer_train.transform(s3_input_train.config['DataSource']['S3DataSource']['S3Uri'], 
                            content_type='text/csv')
print('Waiting for transform job: ' + transformer_train.latest_transform_job.job_name)
transformer_train.wait()
preprocessed_train_path = transformer_train.output_path + transformer_train.latest_transform_job.job_name
print(preprocessed_train_path)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpinyftg_t_algo-1-hos2v_1
algo-1-hos2v_1  | Processing /opt/ml/code
algo-1-hos2v_1  | Building wheels for collected packages: train-preprocessing
algo-1-hos2v_1  |   Building wheel for train-preprocessing (setup.py) ... done
algo-1-hos2v_1  |   Created wheel for train-preprocessing: filename=train_preprocessing-1.0.0-py2.py3-none-any.whl size=9935 sha256=2330f06b019d3a8cc298ecf047af867af9ea8c5ab14c8a7de944aa73553bcc2f
algo-1-hos2v_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-074cd5ih/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-hos2v_1  | Successfully built train-preprocessing
algo-1-hos2v_1  | Installing collected packages: train-preprocessing
algo-1-hos2v_1  | Successfully installed train-preprocessing-1.0.0
algo-1-hos2v_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation

#### Training 전처리된 학습 파일 확인

In [5]:
print(preprocessed_train_path)

s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-09-08-00-2020-08-09-08-01-03-918


In [6]:
! aws s3 ls {preprocessed_train_path} --recursive

2020-08-09 08:01:10    1054526 sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-09-08-00-2020-08-09-08-01-03-918/train.csv.out


In [7]:
preprocessed_train_path_file = os.path.join (preprocessed_train_path, 'train.csv.out')
df_pre_train = pd.read_csv(preprocessed_train_path_file)
df_pre_train.head()


,0.0,0.11941369588439606,-0.5962380254245051,1.744368057672484,0.9789570533336895,-0.028992907038264654,-0.8931854019845896,-0.8017032037830547,-1.9825286353116254,-1.5305589315744583,...,0.0.48,0.0.49,0.0.50,0.0.51,0.0.52,0.0.53,1.0.1,0.0.54,1.0.2,0.0.55
0,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.751486,1.218393,0.231046,-0.756723,0.516833,0.275776,1.043127,-2.138114,0.232814,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


### Preprocessed Validation data (Feature) 만들기

#### 아래는 약 1분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [8])로 바뀔 때까지 기다려 주세요

In [ ]:
# 아웃풋 경로 지정
transform_validation_output_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-validation-output')
# scikit_learn_inferencee_model 에서 Validation Transformer 생성
transformer_validation = scikit_learn_inferencee_model.transformer(
    instance_count=1, 
    instance_type= instance_type,
    assemble_with = 'Line',
    output_path = transform_validation_output_path,
    accept = 'text/csv')
# Preprocess validation input
transformer_validation.transform(s3_input_validation.config['DataSource']['S3DataSource']['S3Uri'], content_type='text/csv')
print('Waiting for transform job: ' + transformer_validation.latest_transform_job.job_name)
transformer_validation.wait()
preprocessed_validation_path = transformer_validation.output_path+transformer_validation.latest_transform_job.job_name
print(preprocessed_validation_path)


In [ ]:
preprocessed_validation_path

In [ ]:
! aws s3 ls {preprocessed_validation_path} --recursive

---
## Train with XGBoost on SageMaker Cloud Instance
아 과정은 위의 전처리된 데이타를 가지고 실제 SageMaker Built-in Algorithm XGBoost를 이용하여 학습을 수행 합니다.<br>
실제의 학습 과정은 SageMaker Cloud Instance에서 실제 학습니 됩니다.


Built-in XGboost 알고리즘 컨테이너를 가져옵니다.

In [ ]:
import boto3

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')

S3에 있는 Train, Validation 전처리된 (Features) 데이타의 경로 및 파일 포맷등을 지정하는 오브젝트를 생성 합니다.

In [ ]:
s3_input_train_processed = sagemaker.session.s3_input(
    preprocessed_train_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("S3 Train input: \n")
print(s3_input_train_processed.config)
s3_input_validation_processed = sagemaker.session.s3_input(
    preprocessed_validation_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("\nS3 Validation input: \n")
print(s3_input_validation_processed.config)

#### 아래는 약 5분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [13])로 바뀔 때까지 기다려 주세요

In [ ]:
sess = sagemaker.Session()
instance_type = 'ml.m4.2xlarge'


xgb = sagemaker.estimator.Estimator(container, # Built-in XGBoost Container
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess
                                   )
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100,
                       )


xgb.fit({'train': s3_input_train_processed, 'validation': s3_input_validation_processed}) 

## Post-processing
- XGBoost에서 나온 결과 값( 0 <= Score <=1) 을 0.5 이하이면 False, 이상이면 True로 변환 함

In [ ]:
# sklearn_preprocessor 로 부터 후처리 모델을 생성 함.(scikit_learn_post_process_model)
# 다만 환경 변수를 바꾸어 후처리용으로 사용 함 ()'TRANSFORM_MODE': 'inverse-label-transform')
# transform_postprocessor_path = 's3://{}/{}/{}/'.format(bucket, prefix, 'transformtrain-postprocessing-output')

# transformer_post_processing = scikit_learn_post_process_model.transformer(
#     instance_count=1, 
#     instance_type='local',
#     assemble_with = 'Line',
#     output_path = transform_postprocessor_path,
#     accept = 'text/csv')

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [ ]:
# 전처리 모델
scikit_learn_pre_process_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'feature-transform'})    
# 학습 모델
xgb_model = xgb.create_model()
# 후처리 모델
scikit_learn_post_process_model = sklearn_preprocessor.create_model(
    env={'TRANSFORM_MODE': 'inverse-label-transform'})

In [ ]:
print("Feature Transformer Model:\n {}".format(scikit_learn_pre_process_model.model_data))
print("\nXGBoost Model:\n {}".format(xgb_model.model_data))
print("\nPost-Processing Model :\n {}".format(scikit_learn_post_process_model.model_data))

전처리 모델의 기타 설정 변수 확인

In [ ]:
print("entry_point: ", scikit_learn_pre_process_model.entry_point)
print("framework_version: ", scikit_learn_pre_process_model.framework_version)
print("env: ", scikit_learn_pre_process_model.env)
print("model_data: ", scikit_learn_pre_process_model.model_data)
print("name: ", scikit_learn_pre_process_model.name)

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3

from time import gmtime, strftime

timestamp_prefix = strftime('$Y-%m-%d-%H-%M-%S', gmtime())

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-model-inference-pipeline-' + timestamp_prefix


sklearn_preprocessor.create_model

pipeline_model = PipelineModel(
    name = model_name,
    role = role,
    models = [
        scikit_learn_pre_process_model,
        xgb_model,
        scikit_learn_post_process_model        
    ]
)

In [ ]:
%%time

instance_type='ml.t2.medium'
endpoint_name= 'churn-model-pipeline-endpoint-' + timestamp_prefix



pipeline_model.deploy(
    initial_instance_count=1, 
    instance_type= instance_type, 
    endpoint_name = endpoint_name    
)

In [ ]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
import sagemaker
sagemaker_session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint = endpoint_name,
    sagemaker_session = sagemaker_session,
    serializer = csv_serializer,
    content_type = CONTENT_TYPE_CSV,
    accept = CONTENT_TYPE_JSON
)



In [ ]:

def make_inference_format(sample):
    instance = str()
    for i, token in enumerate(sample):
        # print(token)
        if i > 0:
            instance = instance  + ',' + str(token) 
        else:
            instance = instance  +  str(token) 
    return instance




In [ ]:
test_df = pd.read_csv("../churn_data/batch_transform_test.csv", header=None)

for i in range(10):
    sample = test_df.iloc[i,:]
    instance = make_inference_format(sample)
    print("instance: \n", instance)

    payload = instance
    churn_result = predictor.predict(payload)
    print("Churn result?: \n", churn_result)
    print("")